## Стандартный пайплайн обучения модели

pykeen предоставляет стандартный интерфейс-[pipeline](https://pykeen.readthedocs.io/en/stable/api/pykeen.pipeline.pipeline.html#pykeen.pipeline.pipeline) для обучения KG embedding алгоритмов на любых датасетах. 

В базовом примере мы обучим TransE (в стандартных настройках) на небольшом датасете Kinships, где обучение не займет больше 4 минут

In [1]:
# Базовый пайплайн обучения модели
from pykeen.pipeline import pipeline
result = pipeline(
    model='TransE',  # строка или класс модели
    dataset='kinships', # строка или класс а датасетом
    training_kwargs={'num_epochs':300}  # аргументы тренировочного цикла
)

C:\Users\El Tuko\anaconda3\envs\pykeen\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No random seed is specified. Setting to 3753162561.
No cuda devices were available. The model runs on CPU
C:\Users\El Tuko\anaconda3\envs\pykeen\lib\site-packages\pykeen\nn\representation.py:369: UserWarning: Directly use Embedding.max_id instead of num_embeddings.
  warnings.warn(f"Directly use {self.__class__.__name__}.max_id instead of num_embeddings.")
C:\Users\El Tuko\anaconda3\envs\pykeen\lib\site-packages\pykeen\nn\representation.py:375: UserWarning: Directly use Embedding.shape instead of num_embeddings.
  warnings.warn(f"Directly use {self.__class__.__name__}.shape instead of num_embeddings.")
Training batches on cpu:  50%|█████     | 17/34 [00:00<00:00, 158.67batch/s]


Training batches on cpu:   0%|          | 0/34 [00:00<?, ?batch/s]


Training batches on cpu:  71%|███████   | 24/34 [00:00<00:00, 63.40batch/s]


Training epochs on cpu:  67%|██████▋   | 202/300 [01:11<00:27,  3.55epoch/s, loss=0.00124, prev_loss=0.00128]


Training epochs on cpu:  85%|████████▌ | 256/300 [01:28<00:11,  3.69epoch/s, loss=0.00119, prev_loss=0.00118]


Training epochs on cpu: 100%|██████████| 300/300 [01:41<00:00,  2.96epoch/s, loss=0.00114, prev_loss=0.0012]
INFO:pykeen.evaluation.evaluator:Currently automatic memory optimization only supports GPUs, but you're using a CPU. Therefore, the batch_size will be set to the default value.
INFO:pykeen.evaluation.evaluator:No evaluation batch_size provided. Setting batch_size to '32'.
Evaluating on cpu: 100%|██████████| 1.07k/1.07k [00:00<00:00, 5.12ktriple/s]
INFO:pykeen.evaluation.evaluator:Evaluation took 0.26s seconds


Основные метрики для оценки предсказательных способностей KGE моделей можно извлечь из объекта-результата. Полный лист включает optimistic, pessimistic, realistic метрики для каждой части head / tail prediction. ([документация](https://pykeen.readthedocs.io/en/stable/tutorial/understanding_evaluation.html)). Оценка, по умолчанию, в фильрованном режиме (filtered setting). Как правило, нас интересует агрегированный результат по обеим частям - для этого мы будем извлекать:
* both - агрегация head / tail предсказаний
* realistic - реалистичный сценарний оценки 

Метрики:
* `hits_at_1` , `hits_at_3`, `hits_at_5`, `hits_at_10`
* `inverse_harmonic_mean_rank` -  он же [Mean Reciprocal Rank (MRR)](https://pykeen.readthedocs.io/en/stable/tutorial/understanding_evaluation.html#inverse-geometric-mean-rank) 

In [2]:
# весь список метрик доступен через to_flat_dict()
# from pprint import pprint
# pprint(result.metric_results.to_flat_dict())
print(f"MRR: {result.metric_results.to_flat_dict()['both.realistic.inverse_harmonic_mean_rank']}")
for k in [1,3,5,10]:
    print(f"Hits@{k} : {result.metric_results.to_flat_dict()['both.realistic.hits_at_'+str(k)]}")

MRR: 0.19941009578314128
Hits@1 : 0.016294227188081937
Hits@3 : 0.24394785847299813
Hits@5 : 0.39664804469273746
Hits@10 : 0.633147113594041


## Имплементация собственных моделей

Собственные модели можно создать двумя способами:
1. Наследовать класс `ERModel` или `EntityRelationEmbeddingModel` в зависимости от выбранного алгоритма. [Документация](https://pykeen.readthedocs.io/en/stable/extending/models_new.html)
2. Создать [interaction function](https://pykeen.readthedocs.io/en/stable/byo/interaction.html) и сгенерировать класс модели через него 

Каждая модель должна переопределить один (или три) метода:

```
def score_hrt(hrt_batch: torch.LongTensor) -> torch.FloatTensor:
    # hrt_batch: триплеты [batch_size, 3], где 0 - субъекты, 1 - предикаты, 2 - объекты
    # output: [batch_size, 1]
    # оценивание всего триплета, стандарт для sLCWA тренировки
    # получение эмбеддингов
    # подсчет score function / interaction function
```

```
def score_t(hr_batch: torch.LongTensor) -> torch.FloatTensor:
    # hr_batch: триплеты [batch_size, 2], где 0 - субъекты, 1 - предикаты
    # output: [batch_size, num_entities]
    # оценивание всех возможных комбинаций с объектами (h,r, ?), стандарт для LCWA тренировки и инференса
    # получение эмбеддингов
    # подсчет score function / interaction function
```

```
def score_h(rt_batch: torch.LongTensor) -> torch.FloatTensor:
    # rt_batch: триплеты [batch_size, 2], где 0 - предикаты, 1 - объекты
    # output: [batch_size, num_entities]
    # оценивание всех возможных комбинаций с субьъектами (?,r,t), стандарт для LCWA тренировки и инференса без инверсных триплетов
    # получение эмбеддингов
    # подсчет score function / interaction function
```

### Базовый класс

In [2]:
import torch
import pykeen
from pykeen.models import ERModel, EntityRelationEmbeddingModel
from pykeen.nn import EmbeddingSpecification

# Базовый класс для моделей в этом упражнении
class CustomModel(EntityRelationEmbeddingModel):
    def __init__(
        self,
        # When defining your class, any hyper-parameters that can be configured should be
        # made as arguments to the __init__() function. When running the pipeline(), these
        # are passed via the ``model_kwargs``.
        embedding_dim: int = 50,
        # All remaining arguments are simply passed through to the parent constructor. If you
        # want access to them, you can name them explicitly. See the pykeen.models.ERModel
        # documentation for a full list
        **kwargs,
    ) -> None:
        # since this is a python class, you can feel free to get creative here. One example of
        # pre-processing is to derive the shape for the relation representation based on the
        # embedding dimension.
        super().__init__(
            # Define the entity representations using the EmbeddingSpecification. By default, each
            # embedding is linear. You can use the ``shape`` kwarg to specify higher dimensional
            # tensor shapes.
            entity_representations=EmbeddingSpecification(
                embedding_dim=embedding_dim,
            ),
            # Define the relation representations the same as the entities
            relation_representations=EmbeddingSpecification(
                embedding_dim=embedding_dim,
            ),
            # All other arguments are passed through, such as the ``triples_factory``, ``loss``,
            # ``preferred_device``, and others. These are all handled by the pipeline() function
            **kwargs,
        )


ImportError: cannot import name 'EmbeddingSpecification' from 'pykeen.nn' (C:\Users\El Tuko\anaconda3\envs\pykeen\lib\site-packages\pykeen\nn\__init__.py)

### TransE

Реализуйте `CustomTransE` по стандартной формуле: 
$$ 
f(h, r, t) = - \|\textbf{e}_h + \textbf{e}_r - \textbf{e}_t\|_{p}
$$

In [18]:
class CustomTransE(CustomModel):
    
    # для sLCWA сценария
    def score_hrt(self, hrt_batch: torch.LongTensor) -> torch.FloatTensor:
        # hrt_batch: триплеты [batch_size, 3], где 0 - субъекты, 1 - предикаты, 2 - объекты
        # output: [batch_size, 1]
        # получение эмбеддингов
        h = self.entity_embeddings(indices=hrt_batch[:, 0])
        r = self.relation_embeddings(indices=hrt_batch[:, 1])
        t = self.entity_embeddings(indices=hrt_batch[:, 2])

        # подсчет scoring function
        # пусть норма p=1
        return -1 * torch.norm(h+r-t, p=1)

        
        #### 
        

    # для LCWA сценария
    def score_t(self, hr_batch: torch.LongTensor) -> torch.FloatTensor:
        # hrt_batch: триплеты [batch_size, 2], где 0 - субъекты, 1 - предикаты
        # output: [batch_size, num_entities]
        # получение эмбеддингов
        h = self.entity_embeddings(indices=hr_batch[:, 0])
        r = self.relation_embeddings(indices=hr_batch[:, 1])
        t = self.entity_embeddings(indices=None)  # None - получение всей матрицы

        # подсчет scoring function для всех объектов (tails)
        # пусть норма p=1
        # будьте аккуратны с tensor shapes
        return -1 * torch.norm(h+r-t, p=1)

        
        #### 
    
    def score_h(self, rt_batch: torch.LongTensor) -> torch.FloatTensor:
        # hrt_batch: триплеты [batch_size, 2], где 0 - предикаты, 1 - объекты
        # output: [batch_size, num_entities]
        # получение эмбеддингов
        h = self.entity_embeddings(indices=None)
        r = self.relation_embeddings(indices=rt_batch[:, 0])
        t = self.entity_embeddings(indices=rt_batch[:, 1])  # None - получение всей матрицы

        # подсчет scoring function для всех субъектов (heads)
        # пусть норма p=1
        # будьте аккуратны с tensor shapes
        return -1 * torch.norm(h+r-t, p=1)

        
        #### 

Проверьте работоспособность модели в sLCWA сценарии. Loss @ 50 epochs ~ 0.0106

In [19]:
result_slcwa = pipeline(
    model=CustomTransE,
    dataset='kinships',
    training_kwargs={'num_epochs':50},
    training_loop='slcwa',
    random_seed=1603073093,
)


INFO:pykeen.pipeline.api:Using device: None
INFO:pykeen.datasets.utils:Loading cached preprocessed dataset from file:///C:/Users/El%20Tuko/.data/pykeen/datasets/kinships/cache/47DEQpj8HBSa-_TImW-5JCeuQeRkm5NM
INFO:pykeen.triples.triples_factory:Loading from file:///C:/Users/El%20Tuko/.data/pykeen/datasets/kinships/cache/47DEQpj8HBSa-_TImW-5JCeuQeRkm5NM/training
INFO:pykeen.triples.triples_factory:Loading from file:///C:/Users/El%20Tuko/.data/pykeen/datasets/kinships/cache/47DEQpj8HBSa-_TImW-5JCeuQeRkm5NM/testing
INFO:pykeen.triples.triples_factory:Loading from file:///C:/Users/El%20Tuko/.data/pykeen/datasets/kinships/cache/47DEQpj8HBSa-_TImW-5JCeuQeRkm5NM/validation


AttributeError: module 'pykeen.nn' has no attribute 'EmbeddingSpecification'

Проверьте работоспособность модели в LCWA сценарии c инверсными гранями. Loss @ 50 epochs ~ 0.0102

In [ ]:
result_lcwa = pipeline(
    model=CustomTransE,
    dataset='kinships',
    dataset_kwargs={'create_inverse_triples': True},
    training_kwargs={'num_epochs':50},
    training_loop='lcwa',
    random_seed=1603073093,
)

Теперь поэкспериментируйте с `num_epochs=500` и разными размерностями `model_kwargs` / `embedding_dim` 
* Натренируйте пайплайны с embedding dimension={50,100,200,500}
* Постройте график зависимости MRR от embedding dimension с двумя моделями (sLCWA и LCWA)


In [ ]:
### Построение графика из результатов пайплайнов ###
### Используйте любые стандартные библиотеки графиков (matplotlib, seaborn)
### ВАШ КОД ЗДЕСЬ ###



###

### DistMult

Реализуйте `CustomDistMult` по стандартной формуле: 
$$ 
f(h, r, t) =  \|\textbf{e}_h \odot \textbf{e}_r \odot \textbf{e}_t\|_{1}
$$

In [ ]:
class CustomDistMult(CustomModel):
    
    # для sLCWA сценария
    def score_hrt(self, hrt_batch: torch.LongTensor) -> torch.FloatTensor:
        # hrt_batch: триплеты [batch_size, 3], где 0 - субъекты, 1 - предикаты, 2 - объекты
        # output: [batch_size, 1]
        # получение эмбеддингов
        h = self.entity_embeddings(indices=hrt_batch[:, 0])
        r = self.relation_embeddings(indices=hrt_batch[:, 1])
        t = self.entity_embeddings(indices=hrt_batch[:, 2])

        # подсчет scoring function
        # вместо нормы можно использовать сумму произведения
        #### ВАШ КОД (1-2 строки) ####


        #### 
        

    # для LCWA сценария
    def score_t(self, hr_batch: torch.LongTensor) -> torch.FloatTensor:
        # hrt_batch: триплеты [batch_size, 2], где 0 - субъекты, 1 - предикаты
        # output: [batch_size, num_entities]
        # получение эмбеддингов
        h = self.entity_embeddings(indices=hr_batch[:, 0])
        r = self.relation_embeddings(indices=hr_batch[:, 1])
        t = self.entity_embeddings(indices=None)  # None - получение всей матрицы

        # подсчет scoring function для всех объектов (tails)
        # вместо нормы можно использовать сумму произведения
        # будьте аккуратны с tensor shapes
        #### ВАШ КОД (1-2 строки) ####

        
        #### 
    

Проверьте работоспособность модели в sLCWA сценарии.

In [ ]:
result_slcwa = pipeline(
    model=CustomDistMult,
    dataset='kinships',
    training_kwargs={'num_epochs':50},
    training_loop='slcwa',
    random_seed=1603073093,
)


Проверьте работоспособность модели в LCWA сценарии c инверсными гранями.

In [ ]:
result_lcwa = pipeline(
    model=CustomDistMult,
    dataset='kinships',
    dataset_kwargs={'create_inverse_triples': True},
    training_kwargs={'num_epochs':50},
    training_loop='lcwa',
    random_seed=1603073093,
)

### ComplEx

Реализуйте `CustomComplEx` по стандартной формуле: 
$$ 
f(h, r, t) =  \text{Re} (\textbf{e}_h \odot \textbf{e}_r \odot \bar{\textbf{e}}_t) 
$$

где $\bar{\mathbf{e}}_t$ - комплексно сопряженное от исходного $\mathbf{e}_t$.

Каждый вектор сущностей и предикатов теперь представляется конкатенацией действительной Re(x) и мнимой Im(x) частей. Поэтому после получения соответствующего вектора из эмбеддинг таблицы его следует разделить на два (например через `torch.chunk()`).

Упраженение на получение конкретной реализации после раскрытия скобок - одно из ДЗ Лекции 7, и здесь можно применить этот результат.

In [ ]:
class CustomComplEx(CustomModel):

    # для упрощения подсчетов можно упаковать взаимодействие в функцию
    def complex_interaction(self, 
                            h_re: torch.FloatTensor,
                            h_im: torch.FloatTensor,
                            r_re: torch.FloatTensor,
                            r_im: torch.FloatTensor,
                            t_re: torch.FloatTensor,
                            t_im: torch.FloatTensor) -> torch.FloatTensor:
        # реализация формулы 
         
    
    # для sLCWA сценария
    def score_hrt(self, hrt_batch: torch.LongTensor) -> torch.FloatTensor:
        # hrt_batch: триплеты [batch_size, 3], где 0 - субъекты, 1 - предикаты, 2 - объекты
        # output: [batch_size, 1]
        # получение эмбеддингов
        h_re, h_im = self.entity_embeddings(indices=hrt_batch[:, 0]).chunk(2, dim=-1)
        r_re, r_im = self.relation_embeddings(indices=hrt_batch[:, 1]).chunk(2, dim=-1)
        t_re, t_im = self.entity_embeddings(indices=hrt_batch[:, 2]).chunk(2, dim=-1)

        # подсчет scoring function
        # используйте выведенную формулу из ДЗ к Лекции 7
        #### ВАШ КОД (5-8 строк) ####


        #### 
        

    # для LCWA сценария
    def score_t(self, hr_batch: torch.LongTensor) -> torch.FloatTensor:
        # hrt_batch: триплеты [batch_size, 2], где 0 - субъекты, 1 - предикаты
        # output: [batch_size, num_entities]
        # получение эмбеддингов
        h_re, h_im = self.entity_embeddings(indices=hr_batch[:, 0]).chunk(2, dim=-1)
        r_re, r_im = self.relation_embeddings(indices=hr_batch[:, 1]).chunk(2, dim=-1)
        t_re, t_im = self.entity_embeddings(indices=None).chunk(2, dim=-1)  # None - получение всей матрицы

        # подсчет scoring function для всех объектов (tails)
        # используйте выведенную формулу из ДЗ к Лекции 7
        # будьте аккуратны с tensor shapes
        #### ВАШ КОД (5-8 строк) ####

        
        #### 
    

Проверьте работоспособность модели в sLCWA сценарии. Размерность комплексных моделей включает действительную и мнимую части, ее можно менять через `model_kwargs` / `embedding_dim`


In [ ]:
result_slcwa = pipeline(
    model=CustomComplEx,
    model_kwarags={'embedding_dim': 200},
    dataset='kinships',
    training_kwargs={'num_epochs':50},
    training_loop='slcwa',
    random_seed=1603073093,
)


Проверьте работоспособность модели в LCWA сценарии c инверсными гранями.

In [ ]:
result_lcwa = pipeline(
    model=CustomComplEx,
    model_kwarags={'embedding_dim': 200},
    dataset='kinships',
    dataset_kwargs={'create_inverse_triples': True},
    training_kwargs={'num_epochs':50},
    training_loop='lcwa',
    random_seed=1603073093,
)

### RotatE

Реализуйте `CustomRotatE` по стандартной формуле: 
$$ 
f(h, r, t) =  - \| h \odot r - t \|
$$

где $h \odot r$ - вращение вектора в комплексном пространстве.

Каждый вектор сущностей и предикатов теперь представляется конкатенацией действительной Re(x) и мнимой Im(x) частей. Поэтому после получения соответствующего вектора из эмбеддинг таблицы его следует разделить на два (например через `torch.chunk()`).

Помимо этого, векторы предикатов обозначают части оператора вращения. Мы используем дополнительные `initializer` и `complex_normalize` чтобы нормировать модуль комплексного числа к единице.

Упраженение на получение конкретной реализации комплексного вращения - одно из ДЗ Лекции 7, и здесь можно применить этот результат.

In [ ]:
from pykeen.nn.init import init_phases, xavier_uniform_
from pykeen.nn import EmbeddingSpecification
from pykeen.typing import Constrainer, Hint, Initializer
from pykeen.utils import complex_normalize

class CustomRotatE(CustomModel):

    def __init__(
        self,
        *,
        embedding_dim: int = 200,
        entity_initializer: Hint[Initializer] = xavier_uniform_,
        relation_initializer: Hint[Initializer] = init_phases,
        relation_constrainer: Hint[Constrainer] = complex_normalize,
        **kwargs,
    ) -> None:
        super().__init__(
            entity_representations=EmbeddingSpecification(
                embedding_dim=embedding_dim,
                initializer=entity_initializer,
                dtype=torch.cfloat,
            ),
            relation_representations=EmbeddingSpecification(
                embedding_dim=embedding_dim,
                initializer=relation_initializer,
                constrainer=relation_constrainer,
                dtype=torch.cfloat,
            ),
            **kwargs,
        )
        self.real_embedding_dim = embedding_dim

    # для упрощения подсчетов можно упаковать взаимодействие в функцию
    def rotate_interaction(self, 
                            h_re: torch.FloatTensor,
                            h_im: torch.FloatTensor,
                            r_re: torch.FloatTensor,
                            r_im: torch.FloatTensor,
                            t_re: torch.FloatTensor,
                            t_im: torch.FloatTensor) -> torch.FloatTensor:
        # реализация формулы 
                 
    
    # для sLCWA сценария
    def score_hrt(self, hrt_batch: torch.LongTensor) -> torch.FloatTensor:
        # hrt_batch: триплеты [batch_size, 3], где 0 - субъекты, 1 - предикаты, 2 - объекты
        # output: [batch_size, 1]
        # получение эмбеддингов
        h_re, h_im = self.entity_embeddings(indices=hrt_batch[:, 0]).chunk(2, dim=-1)
        r_re, r_im = self.relation_embeddings(indices=hrt_batch[:, 1]).chunk(2, dim=-1)
        t_re, t_im = self.entity_embeddings(indices=hrt_batch[:, 2]).chunk(2, dim=-1)

        # подсчет scoring function
        # используйте выведенную формулу из ДЗ к Лекции 7
        # пусть норма p=1
        #### ВАШ КОД (5-8 строк) ####
        

        #### 
        

    # для LCWA сценария
    def score_t(self, hr_batch: torch.LongTensor) -> torch.FloatTensor:
        # hrt_batch: триплеты [batch_size, 2], где 0 - субъекты, 1 - предикаты
        # output: [batch_size, num_entities]
        # получение эмбеддингов
        h_re, h_im = self.entity_embeddings(indices=hr_batch[:, 0]).chunk(2, dim=-1)
        r_re, r_im = self.relation_embeddings(indices=hr_batch[:, 1]).chunk(2, dim=-1)
        t_re, t_im = self.entity_embeddings(indices=None).chunk(2, dim=-1)  # None - получение всей матрицы

        # подсчет scoring function для всех объектов (tails)
        # используйте выведенную формулу из ДЗ к Лекции 7
        # пусть норма p=1
        # будьте аккуратны с tensor shapes
        #### ВАШ КОД (5-8 строк) ####
        
        
        #### 
    

Проверьте работоспособность модели в sLCWA сценарии. Размерность комплексных моделей включает действительную и мнимую части, ее можно менять через `model_kwargs` / `embedding_dim`


In [ ]:
result_slcwa = pipeline(
    model=CustomRotatE,
    model_kwarags={'embedding_dim': 200},
    dataset='kinships',
    training_kwargs={'num_epochs':50},
    training_loop='slcwa',
    random_seed=1603073093,
)


Проверьте работоспособность модели в LCWA сценарии c инверсными гранями.

In [ ]:
result_lcwa = pipeline(
    model=CustomRotatE,
    model_kwarags={'embedding_dim': 200},
    dataset='kinships',
    dataset_kwargs={'create_inverse_triples': True},
    training_kwargs={'num_epochs':50},
    training_loop='lcwa',
    random_seed=1603073093,
)

### Визуализация

Используйте pipeline чтобы натренировать несколько моделей в разных настройках `embedding_dim`, `num_epochs` и др. 
Визуализируйте полученные эмбеддинги (полной размерности для TransE и DistMult, и только векторы действительной части комплексного числа для ComplEx и RotatE) с помощью стандартного sklearn tSNE - находите ли вы кластеры сущностей?


In [ ]:
from sklearn.manifold import TSNE
# получение матрицы эмбеддингов из результата тренировки
# <RESULT_OBJ>.model.entity_embeddings().shape

In [ ]:
z = <YOUR_RESULT_OBJ>.model.entity_embeddings().cpu().detach().numpy()  # используйте слайсинг для получения первой половины колонок матрицы в случае комплексных чисел
xs, ys = zip(*TSNE().fit_transform(z))

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.scatter(xs, ys)
plt.show()